## BIOENG-310: Neuroscience Foundations for Engineers

Notebook created by Yingtian Tang, edited by Alejandro Rodriguez Guajardo.

# Week 7: Graded exercise - Align to IT activity

This week, we will test your understanding of what we've learned.
We are shifting focus to data from the **inferior temporal gyrus (IT)** in monkeys. IT is a high-level region in the ventral visual stream, where neural populations encode object categories.
In contrast, **V1**, which we previously worked with, is an early-stage area in the stream. The processing hierarchy from V1 to V2, V4, and IT progressively builds computations that support vision—from edge and color detection to object categorization.

The notebook consists of the following sessions:
- **Data visualization**:
  * Visualize stimulus samples (1 points)
  * Visualize stimulus-averaged neural response time series (2 points)
  * Visualize time-averaged neural responses (2 points)
- **Decode object category from IT activity** (3 points)
- **Model alignment using RDM**
  * Align Gabor filtering to IT activity (4 points)
  * Align AlexNet to IT activity (3 points)

The whole exercise is worth 15 points (15% of your final grade). You can navigate through these sections using the `Table of contents` on the left (in Google Colab).

Please complete the tasks described in each section.
You may use multiple consecutive Python code blocks to complete the tasks, but ensure they remain within the same section as the corresponding task.

Enjoy the challenge! 🏆

**Before you begin**, execute the following block to install the required packages. (The following code might require you to *restart runtime*, which is fine)



In [ ]:
!pip install jupyter brainscore-vision matplotlib

In [ ]:
%matplotlib inline
import cv2
import numpy as np
import xarray as xr
import brainscore_vision
from matplotlib import pyplot, image

data = brainscore_vision.load_dataset('MajajHong2015.temporal.public')

# we'll focus on only IT recordings in this exercise
it_data = data.sel(region='IT')

# to make our life easier, we only work on a subset of the data this time
# this is going to take some time to load
stimulus_ids = it_data['stimulus_id'].values
target_stimuli = np.unique(stimulus_ids)[:135]
stimulus_mask = [stimulus_id in target_stimuli for stimulus_id in stimulus_ids]
it_data = it_data.isel(presentation=stimulus_mask)
stimulus_set = it_data.stimulus_set
stimulus_set = stimulus_set[stimulus_set['stimulus_id'].isin(target_stimuli)]
it_data.attrs["stimulus_set"] = stimulus_set

it_data

In [4]:
# load the it_data to memory
it_data = it_data.load()

### Data visualization

In this section, please try to visualize the following aspects of the data:
1. **Stimulus samples:** example images shown to the monkeys.
2. **Stimulus-averaged neural response time series:** firing rate time series averaged across all stimuli for each neural site.
3. **Time-averaged neural responses:** mean firing rates computed over the 70~170 ms window for each neural site and each stimulus.

#### Visualize stimulus samples

Please visualize the first 10 stimuli (images) from the stimulus set in grayscale.

**Hint:** you can adapt the code from *exercise5 : Stimuli* for this purpose.

In [ ]:
'''%% Visualize the first 10 stimuli (images) from the stimulus set in grayscale. %%'''

#### Visualize stimulus-averaged neural response time series

Please visualize the following:

- **Per neural site:** The firing rate time series averaged across all stimuli for each neural site.
- **Overall mean:** The average time series computed across all neural sites.

For each stimulus, you should average across repetitions.

**Hint:** you can adapt codes from *exercise5 : Data visualization*.

In [ ]:
'''%% Visualize the per-neural-site and overall-mean firing rate time series, averaged across all stimuli. %%'''

#### Visualize time-averaged neural responses

Please visualize a colormap matrix representing neural responses, with:

- **Columns:** All stimuli, ordered according to `stimulus_set`.
- **Rows:** All neural sites, ordered as in `it_data`.
- **Color Encoding:** Mean firing rates within the 70~170 ms window. Use `'Greens'` as the `cmap`.

For each stimulus, you should average across repetitions.

**Hint:** you can adapt the code from *exercise5 : Time-averaging*.

In [ ]:
'''%% Visualize a colormap of shape [#stimuli x #neuroid], with color indicating the averaged firing rate from 70 to 170ms. %%'''

### Decode object category from IT activity

In this section, please try to decode the object category (the `category_name` coord in the `it_data`) using the time-averaged neural responses from the last section. There are in total 8 classes and 3200 different images in this case. **Please report the validation accuracy and compare it to the theoretical chance performance.**

**Hint:** you can adapt codes from *exercise5 : Predicting/decoding the texture types*.

Please use the train-test split and regressor given below. Also, normalize the inputs during the regression.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

SEED=3
stimulus_ids = stimulus_set['stimulus_id'].values
train_stimuli, val_stimuli = train_test_split(stimulus_ids, train_size=0.8, random_state=SEED)
linear_readout = LogisticRegression(random_state=SEED)

In [ ]:
'''%% Report the validation accuracy of decoding the object category from IT activity and the chance performance of this classification. %%'''

### Model alignment using RDM

In this section, please align two different models to the IT activations using RDM. These models are:

- **Gabor filtering:** a set of simple filters for edge detection.
- **AlexNet:** an artificial deep neural network trained on the object recognition task. We will provide its activations for the stimuli as an `xarray`.

#### Align Gabor filtering to IT activity

Please compute the RDM alignment between activations from a set of Gabor filters and those from monkey IT. As a reminder, a Gabor filter is:

$$g(x,y) = s(x,y)~w(x,y)$$
where
$$ s(x,y) = cos\left( \frac{2\pi x'}{\lambda} \right) $$
$$ w(x,y) = exp \left( - \frac{{x'}^2 + \gamma^2 {y'}^2}{2 \sigma^2} \right) $$
and
$$x' = cos\theta~x + sin\theta~y$$
$$y' = -sin\theta~x + cos\theta~y$$

Please use the following parameters for the Gabor filters:
1. `kernel_size` set to 25
2. $\theta=90, 45, 0, -45°$ (convert them to radians)
3. $\lambda= 0.1, 0.3, 1, 3, 9$
4. $\gamma= 0.25, 0.5, 1, 2, 4$
5. $\sigma= 0.8\lambda$

**Hint:** you can adapt codes from *exercise6 : Alignment between Gabor filtering and monkey V1 activity* and other parts of *exercise6*. \\
**Hint:** do you think the Gabor filtering will align to the IT?

The following are some helper functions.

In [39]:
# generate a square mesh grid of certain size
def meshgrid(kernel_size):
    return np.mgrid[:kernel_size, :kernel_size] - (kernel_size // 2)

# we use an efficient convolution implementation in opencv
def convolve2d(img, filter):
    return cv2.filter2D(img, -1, filter)

# to speed up computations, we downsample the images
def downsample(img, factor=4):
    return img[..., ::factor, ::factor]

# you need to pass in 'gabor_filter_bank', which is a set of Gabor filters
def gabor_filtering(img, gabor_filter_bank):
  # normalize the image
  img = (img - 0.5) / 0.5
  gabor_outputs = []
  for gabor_filter in gabor_filter_bank:
      gabor_outputs.append(downsample(convolve2d(img, gabor_filter)))

  gabor_outputs = np.array(gabor_outputs).flatten()

  # we add a simple nonlinearity into the system
  # by making only "neurons" with positive potential fire
  gabor_outputs[gabor_outputs<0] = 0

  return gabor_outputs

# you need to pass in the RDMs of Gabor filtering and IT data
def rdm_metric(rdm1, rdm2):
    rdm1_upper_tri = rdm1[np.triu_indices(rdm1.shape[0], k=1)]
    rdm2_upper_tri = rdm2[np.triu_indices(rdm2.shape[0], k=1)]
    return np.corrcoef(rdm1_upper_tri, rdm2_upper_tri)[0, 1]

In [ ]:
'''%% Report the RDM alignment between the Gabor filtering and IT activity %%'''

#### Align AlexNet to IT activity

Please use the AlexNet activations to align to the IT using RDM. Report the RDM alignment score.

[AlexNet](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf) is a hierarchical artificial neural network that includes multiple layers of convolutions, nonlinear activation functions, and a multi-layer classifier for predicting the object category. The weights of the convolutions are learned -- they are optimized so that the final layer can predict the object category of the input image. This is very different from Gabor filters, where the weights (or parameters) are pre-determined.

Here, we give you the activations from the last convolution layer of AlexNet.

In [ ]:
from brainio.assemblies import NeuroidAssembly

# download
url = "https://github.com/epflneuroailab/bioeng-310/raw/refs/heads/main/week7/alexnet_activations.nc"
!wget -O alexnet_activations.nc $url

alexnet_assembly = NeuroidAssembly(xr.open_dataarray('alexnet_activations.nc'))

In [ ]:
'''%% Report the RDM alignment between the AlexNet and IT activity %%'''